In [5]:
from random import randint

## Functions review
Functions are a way to encapsulate parts of a program.
Here, you'll put together everything we've learned so far.

In [1]:
# First, define a function that calculates the next number in the Collatz chain.
def nextCollatzNumber(initialNum):
    """Calculates the next Collatz number after initialNum.

    The Collatz numbers are given by

    next = prev // 2      if prev is even
           prev * 3 + 1   if prev is odd
    """
    if (initialNum % 2 == 0):
        return initialNum // 2
    else:
        return (initialNum * 3) + 1

# We can now generate the collatz path for 9.
def collatz9():
    n = 9
    pathLength = 0
    print(pathLength , n)
    while (n != 1):
        n = nextCollatzNumber(n)
        pathLength = pathLength + 1
        print(pathLength, n)
    
# We could also make that loop a function itself.
# This function will calculate the length of the collatz path of n.
def collatzPath(n):
    """How many steps does it take for the Collatz path for n to reach 1?

    :param n: The initial number, which must be positive.
    :return: An integer giving the number of steps it takes for the Collatz
        sequence to reach 1.
    """
    if(n < 1): #A little safety feature: If it's called with n<1,
               #safely return without hanging forever. 
        return -1
    pathLength = 0
    while (n != 1):
        n = nextCollatzNumber(n)
        pathLength = pathLength + 1
    return pathLength

In [ ]:
print("The path length of 10 is ",collatzPath(10))

# Say I want to find the largest number randint(1,10000)
# returns when I run it n times.
def largestRand(n):
    """Samples n random numbers from 1 to 10,000 and returns the largest."""
    largest = 0
    for i in range(n):
        a = randint(1,10000)
        if(a > largest):
            largest = a
    return largest

In [5]:
print("after 100 passes, the largest number returned was ",largestRand(100))

after 100 passes, the largest number returned was  9955


## Decomposition

Fundamentally, functions provide a way to encapsulate one logical chunk of
a program. Consider a function to perform Monte Carlo integration.
(see http://en.wikipedia.org/wiki/Monte_Carlo_integration, we're sampling
a function a bunch of times, and the area under that function is its
average value times the width of the interval.)
There are essentially three components to such a program:
1. The main controller that operates the other bits and displays output
2. The integrator that samples the function and determines the area under it.
3. The function being sampled.

On the one hand, we can implement it as one big function.
This program takes as arguments the number of samples, the function being
integrated, and the bounds of the integration.

In [3]:
def monteCarloSingle(numSamples, fname, start, stop):
    """Integrates the function given by fname from start to stop.

    Uses random sampling to calculate

    ⌠stop
    |       f(x) dx
    ⌡start

    :param numSamples: How many random samples should be used?
    :param fname: The function you want to integrate. Options are:
         's', meaning sin(x)
         'c', meaning cos(x)
         'e', meaning e^x
    :param start: The start point of the integral
    :param stop: The end point of the integral
    :return: Nothing, but does print its result.
    """
    area=0
    for i in range(numSamples):
        #Get a random number between the start and stop range:
        randomNum = randint(0, 1000000) #get a random number
        #and convert that integer to a number in our sample domain
        x = randomNum / 1000000 * (start - stop) + start
        if(fname == 's'): #calculate the sine of the number with a taylor series
            funVal = x - 1/6*x**3 + 1/120*x**5 - 1/5040*x**7 + 1/362880*x**9
        if(fname == 'c'): #calculate the cosine.
            funVal = 1-1/2*x**2 + 1/24*x**4 - 1/720*x**6+1/40320*x**8
        if(fname == 'e'): #calculate e^x.
            funVal = 1+x+1/2*x**2 + 1/6*x**3 + 1/24*x**4 + 1/120*x**5+1/720*x**6+1/5040*x**7
        area = area + funVal / numSamples * (stop - start)
    print("the area between ", start, " and ", stop, " is ", area)

In [6]:
#Okay, now I can run it:
monteCarloSingle(10000, 's', 0, 1)

the area between  0  and  1  is  -0.4593181760896061


Uh-oh. It's negative. There must be a bug somewhere in the code.
But the integrator is one huge block of code;
I have to analyze the whole thing at once. The problem may be in the Taylor
series, it may be in the accumulation of the area, it could be anywhere.

In [7]:
#Now, we can decompose the program using functions:
#First, the main controller:
def monteCarloDecomposed(numSamples, fname, start, stop):
    """See monteCarloSingle"""
    area = mcIntegrate(numSamples, fname, start, stop)
    print("the area between ",start," and ", stop, " is ",area)

#Now, the integrator:
def mcIntegrate(numSamples, fname, start, stop):
    """Actually runs the integration. Same as monteCarloSingle, but returns the area."""
    area = 0
    for i in range(numSamples):
        #First, get a random number in the desired range.
        x = randomInRange(start, stop)
        #then evaluate the function at that location
        funVal = evaluateFunction(fname, x)
        area = area + funVal / numSamples * (stop - start)
    return area

#Okay, so now I need to write randomInRange and evaluateFunction.
def randomInRange(start, stop):
    """Returns a random number between start and stop."""
    randomNum = randint(0, 1000000)
    retVal = randomNum / 1000000 * (start - stop) + start
    return retVal

def evaluateFunction(fname, x):
    """Uses a Taylor expansion to approximate f(x)

    :param fname: A one-character code naming the function. Options are:
         's', meaning sin(x)
         'c', meaning cos(x)
         'e', meaning e^x
    :param x: The x coordinate you want to evaluate the function at.
    :return: The value of f(x).
    """
    if(fname == 's'): 
        funVal = x - 1/6*x**3 + 1/120*x**5 - 1/5040*x**7 + 1/362880*x**9
    if(fname == 'c'): 
        funVal = 1-1/2*x**2 + 1/24*x**4 - 1/720*x**6+1/40320*x**8
    if(fname == 'e'): 
        funVal = 1+x+1/2*x**2 + 1/6*x**3 + 1/24*x**4 + 1/120*x**5+1/720*x**6+1/5040*x**7
    return funVal

In [8]:
monteCarloDecomposed(10000, 's', 0,1)

the area between  0  and  1  is  -0.46305714243895474


Okay, so the bug is still there. But I can play with the components to see
which one has a bug. I think it's the Taylor expansion. $sin(0.5) = 0.479$, so let's test
my implementation of `evaluateFunction`:

In [9]:
evaluateFunction('s', 0.5)

0.4794255386164159

Okay, so the bug doesn't seem to be in `evaluateFunction`.
Perhaps it's in `randomInRange`?

In [11]:
randomInRange(0,1)

-0.119604

A-ha! I have now narrowed down the bug to one line of code. 

## Exercises

1. Of the numbers 1 to 95 (including 95), which has the largest Collatz path?

2. Fix the decomposed Monte-carlo integrator. 

3. A ball is dropped from an initial height H0.
Every time it bounces, it loses some energy. It bounces to 75% of its drop
height on each bounce. How many bounces will it take for the bounce to be
less than 1 mm if the ball is dropped from 2m?

4. Consider a bed file containing regions of interest in a genome.
An analysis requires that regions within N bases of the edge of a
chromosome be removed. You've been tasked with writing a function
that does this. It should take the name of the input bed file as
a string, the name of the output as a string, a name of a file containing
chromosome size information as a string, and a number N giving the padding
boundary.
Write documentation for such a function.
(You don't actually have to write the function, just the documentation.)

5. Given two strings of DNA, write a function that does the following:
    1. Make sure that both strings are valid DNA.
    2. Make sure they have the same length.
    3. Calculate the number of letters that are different between them.
   
    The function should return the number of differences, or -1 if there was a problem with a or b.
    (Incidentally, this metric is called the Hamming weight, and is used in
    both bioinformatics and spell-checkers.)
    Include documentation.

6. Write a function that determines if a sequence of DNA contains a start codon (`ATG`). It should return True or False. Include documentation.

7. (This question is quite a bit tougher than the previous ones, and is optional.)

Write a function that takes a string of DNA and searches it for potential ORFs.
For this problem, an ORF is a string of DNA that starts with a start codon (`ATG`)
and ends with a stop codon (`TAG`, `TGA`, or `TAA`) in the same frame, and that does not
contain an in-frame stop codon other than at the end.
It should print out all possible ORFs based on their start and stop coordinates.
Include documentation with all of your functions.

For example, the sequence 
```
CCGCCGATGCCGCCGCCGTGACCTGAGTGATGCCGCTGACGTAA
01234567890123456789012345678901234567890123
```
has three possible frames. I've marked the start (S) and stop (X) codons
in these frames:
```
         S               X           X           X           
CCG CCG ATG CCG CCG CCG TGA CCT GAG TGA TGC CGC TGA CGT AA
                                X       S               X
CC GCC GAT GCC GCC GCC GTG ACC TGA GTG ATG CCG CTG ACG TAA
                                                                                                                      
C CGC CGA TGC CGC CGC CGT GAC CTG AGT GAT GCC GCT GAC GTA A
```

The first two sequences contain possible ORFs. 
Your program should print the start and stop codon positions
for these ORFs. For example,
`locateORFs("CCGCCGATGCCGCCGCCGTGACCTGAGTGATGCCGCTGACGTAA")`
might print 
```
6 18
29 41
```
Where those numbers are the indexes of the start and stop codons.

This is a TOUGH problem! I recommend you break it up into chunks,
like maybe one function to break a string into codons, one to
check to see if a string is a start (or stop) codon, and so on.